<a href="https://colab.research.google.com/github/Sanskar6877/BTP/blob/main/Implicit_Hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_hub as hub
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files

# Choose the file from your local machine
uploaded = files.upload()


Saving IH_train.csv to IH_train.csv


In [ ]:
df = pd.read_csv("IH_train.csv", encoding='unicode_escape')

In [ ]:
df["class"].value_counts()

not_hate         7972
implicit_hate    4251
explicit_hate     665
Name: class, dtype: int64

In [ ]:
df['hate'] = df['class'].apply(lambda x:0 if x=='not_hate' else 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df['post'], df['hate'],
                                                     stratify = df['hate'])

In [ ]:
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.5 MB/s eta 0:00:00


In [ ]:
import tensorflow_text as text

In [ ]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)

In [ ]:
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Inputs (InputLayer)         [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['Inputs[0][0]']              
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [ ]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]

model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [ ]:
history = model.fit(X_train, y_train, epochs = 10)


Epoch 1/10
303/303 [==============================] - 109s 314ms/step - loss: 0.6666 - accuracy: 0.6099 - precision: 0.4728 - recall: 0.1983
Epoch 2/10
303/303 [==============================] - 98s 323ms/step - loss: 0.6466 - accuracy: 0.6246 - precision: 0.5180 - recall: 0.2259
Epoch 3/10
303/303 [==============================] - 98s 324ms/step - loss: 0.6402 - accuracy: 0.6355 - precision: 0.5427 - recall: 0.2826
Epoch 4/10
303/303 [==============================] - 100s 329ms/step - loss: 0.6365 - accuracy: 0.6376 - precision: 0.5473 - recall: 0.2886
Epoch 5/10
303/303 [==============================] - 100s 330ms/step - loss: 0.6313 - accuracy: 0.6418 - precision: 0.5556 - recall: 0.3049
Epoch 6/10
303/303 [==============================] - 100s 329ms/step - loss: 0.6251 - accuracy: 0.6458 - precision: 0.5633 - recall: 0.3173
Epoch 7/10
303/303 [==============================] - 99s 328ms/step - loss: 0.6259 - accuracy: 0.6481 - precision: 0.5657 - recall: 0.3339
Epoch 8/10
303/3

In [ ]:
model.evaluate(X_test,y_test)

101/101 [==============================] - 33s 320ms/step - loss: 0.6023 - accuracy: 0.6831 - precision: 0.6086 - recall: 0.4744


[0.6023159027099609, 0.68311607837677, 0.6085594892501831, 0.4743694067001343]